<center>
    <h1>Fine-tuning with Few-Shot Learning</h1>
</center>

Traditional fine-tuning methods:

- Require large amounts of labeled training data
- Can be computationally intensive and time-consuming
- Risk overfitting on small datasets
- May need extensive hyperparameter tuning

# Brief recap of Few-Shot Learning

## What is Few-Shot Learning?

- Few-shot learning (FSL) is a machine learning approach where models learn to recognize new patterns or classes from very few examples, typically 1-5 samples per class. 

- Think of it like a human being able to recognize a new type of fruit after seeing just a couple of examples, by drawing on their prior knowledge of similar fruits.

Key characteristics:
- Learns from minimal examples (1-5 samples per class)
- Leverages prior knowledge from similar tasks
- Adapts quickly to new scenarios
- Maintains good generalization

## The Problem Few-Shot Learning Solves

### 1. Data Scarcity

Real-world ML applications face significant data challenges:

* **Limited Training Examples**
  - Many scenarios have very few examples
  - Rare cases or events are hard to collect
  - New classes emerge frequently
  - Data collection is expensive or time-consuming

* **Data Collection Constraints**
  - Privacy concerns limit data availability
  - Legal restrictions on data gathering
  - Time-sensitive scenarios
  - Resource limitations

* **Domain-Specific Challenges**
  - Specialized knowledge required
  - Expert annotation costs
  - Evolving categories
  - Rapid deployment needs

* **Real-world Limitations**
  - Imbalanced class distributions
  - High data acquisition costs
  - Time constraints
  - Resource constraints

### 2. Adaptation Requirements

Modern ML systems need quick adaptation capabilities:

* **Fast Learning**
  - Quick response to new situations
  - Rapid prototype development
  - Immediate deployment needs
  - Market responsiveness

* **Efficient Transfer**
  - Knowledge reuse
  - Cross-domain adaptation
  - Skill transfer
  - Experience leverage

* **Resource Optimization**
  - Minimal data requirements
  - Reduced computation needs
  - Lower energy consumption
  - Faster deployment

* **Practical Constraints**
  - Budget limitations
  - Time restrictions
  - Hardware constraints
  - Deployment requirements

### 3. Model Flexibility

Systems need to be adaptable:

* **Quick Adaptation**
  - New task learning
  - Domain transfer
  - Concept drift handling
  - Dynamic environments

* **Generalization**
  - Cross-domain performance
  - Novel situation handling
  - Robust predictions
  - Transfer learning

## How does Few Shot Learning work?

Few-shot learning combines several key concepts to achieve efficient learning from limited data:

### Core Concepts

**1. Meta-Learning**
- Learns how to learn from few examples
- Develops adaptable feature representations
- Creates flexible learning strategies

**2. Transfer Learning**
- Leverages knowledge from related tasks
- Adapts pre-trained models
- Utilizes domain knowledge

**3. Metric Learning**
- Learns similarity measures
- Creates embedded space representations
- Enables comparison-based learning

### Core Components

**1. Support Set**
- Contains few examples per class
- Provides reference patterns
- Establishes class prototypes

**2. Query Set**
- New examples to classify
- Test cases for evaluation
- Unknown samples for prediction

**3. Episode Training**
- Simulates few-shot scenarios
- Creates learning episodes
- Builds robust representations

### Implementation Approach

**1. Pre-training Phase**
- Model learns on large dataset
- Develops feature extraction
- Creates base knowledge

**2. Fine-tuning Phase**
- Adapts to new tasks
- Uses few examples
- Maintains performance

**3. Inference Process**
- Compares new samples
- Uses learned metrics
- Makes predictions

## Applications

1. **Computer Vision**
   - Face recognition
   - Object detection
   - Medical imaging

2. **Natural Language Processing**
   - Intent classification
   - Named entity recognition
   - Translation for low-resource languages

3. **Robotics**
   - Task learning
   - Behavior adaptation
   - Skill transfer

# Practical Project: Restaurant Review Classifier

In this project, we'll implement few-shot learning to classify restaurant reviews. We'll use OpenAI's API for demonstration, but the concepts apply to any LLM.

## Project Overview

- Task: Classify restaurant reviews into detailed categories
- Method: Few-shot learning with 3 examples per category
- Categories: Food Quality, Service, Ambiance, Value

## Implementation

### 1. Setup and Dependencies

In [ ]:
import openai
import pandas as pd
from typing import List, Dict
import os
from dotenv import load_dotenv

load_dotenv()

In [ ]:
# Set up the client
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

### 2. Define Example Database

In [5]:
few_shot_examples = {
    "food_quality": [
        {"review": "The pasta was perfectly al dente and the sauce was rich and flavorful.", 
            "category": "Food Quality: Positive - Mentions cooking technique and taste"},
        {"review": "My steak was overcooked and dry, not what I expected.", 
            "category": "Food Quality: Negative - Mentions cooking issues"},
        {"review": "Fresh ingredients but the seasoning was a bit overwhelming.", 
            "category": "Food Quality: Mixed - Acknowledges quality but notes issues"}
    ],
    "service": [
        {"review": "Our waiter was attentive and provided excellent recommendations.", 
            "category": "Service: Positive - Mentions staff helpfulness"},
        {"review": "Had to wait 45 minutes for our appetizers, no explanation given.", 
            "category": "Service: Negative - Mentions timing issues"},
        {"review": "Friendly staff but seemed understaffed during peak hours.", 
            "category": "Service: Mixed - Notes good attitude but operational issues"}
    ],
    "value": [
        {"review": "Large portions and reasonable prices, great deal for the quality.", 
            "category": "Value: Positive - Mentions portion size and price"},
        {"review": "Small portions for the high price point, felt overpriced.", 
            "category": "Value: Negative - Comments on poor price-to-portion ratio"},
        {"review": "Slightly pricey but the quality justifies the cost.", 
            "category": "Value: Mixed - Acknowledges cost but justifies value"}
    ]
}

### 3. Prompt Construction Function

In [6]:
def create_few_shot_prompt(examples: Dict, new_review: str) -> str:
    """
    Create a few-shot prompt using examples and a new review.
    
    Args:
        examples (Dict): Dictionary containing categorized example reviews
        new_review (str): New review to be classified
    
    Returns:
        str: Formatted prompt for the LLM
    """
    prompt = "Task: Classify restaurant reviews into categories (Food Quality, Service, or Value) and determine the sentiment (Positive, Negative, or Mixed). Provide a brief explanation for the classification.\n\n"
    prompt += "Examples:\n\n"
    
    # Add examples from each category
    for category, category_examples in examples.items():
        for example in category_examples:
            prompt += f"Review: {example['review']}\n"
            prompt += f"Classification: {example['category']}\n\n"
    
    # Add new review
    prompt += f"Review: {new_review}\n"
    prompt += "Classification:"
    
    return prompt

### 4. Classification Function

In [7]:
def classify_review(review: str, examples: Dict) -> str:
    """
    Classify a restaurant review using few-shot learning.
    
    Args:
        review (str): Review text to classify
        examples (Dict): Few-shot examples dictionary
    
    Returns:
        str: Classification result
    """
    prompt = create_few_shot_prompt(examples, review)
    
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful restaurant review classifier."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=100
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error in classification: {str(e)}"

### 5. Testing Function

In [8]:
def test_classifier(test_reviews: List[str], examples: Dict) -> pd.DataFrame:
    """
    Test the classifier on multiple reviews and return results.
    
    Args:
        test_reviews (List[str]): List of reviews to classify
        examples (Dict): Few-shot examples dictionary
    
    Returns:
        pd.DataFrame: Results of classification
    """
    results = []
    for review in test_reviews:
        classification = classify_review(review, examples)
        results.append({
            'review': review,
            'classification': classification
        })
    
    return pd.DataFrame(results)

### 6. Example Usage

In [ ]:
test_reviews = [
    "The flavors were amazing but $40 for a small plate seemed excessive.",
    "Our server was incredibly helpful and checked on us regularly.",
    "Burnt food and rude service, definitely not worth the price.",
    "The new chef has really improved the menu, every dish was perfect."
]

# Run classification
results_df = test_classifier(test_reviews, few_shot_examples)
print(results_df)

In [ ]:
results_df['classification'][0]